In [60]:
import pandas as pd
import numpy as np
import math

In [61]:
temp_Quito = pd.read_csv('TemperaturasRegistradas/temperature_Quito_celsius.csv')
temp_Oslo = pd.read_csv('TemperaturasRegistradas/temperature_Oslo_celsius.csv')
temp_Melbourne = pd.read_csv('TemperaturasRegistradas/temperature_Melbourne_celsius.csv')

In [62]:
promQuito = temp_Quito.sum()/temp_Quito.count()
promOslo = temp_Oslo.sum()/temp_Oslo.count()
promMelbourne = temp_Melbourne.sum()/temp_Melbourne.count()

In [63]:
i=0
for v in temp_Quito.values.flatten():
    i += (v-promQuito)**2
varianzaQuito = i / (temp_Quito.count()-1)
DesvQuito = math.sqrt(float(varianzaQuito.iloc[0]))
print(DesvQuito)

j=0
for v in temp_Oslo.values.flatten():
    j += (v-promOslo)**2
varianzaOslo = j / (temp_Oslo.count()-1)
DesvOslo = math.sqrt(float(varianzaOslo.iloc[0]))
print(DesvOslo)

k=0
for v in temp_Melbourne.values.flatten():
    k += (v-promMelbourne)**2
varianzaMelbourne = k / (temp_Melbourne.count()-1)
DesvMelbourne = math.sqrt(float(varianzaMelbourne.iloc[0]))
print(DesvMelbourne)



1.301690430933042
9.46169397382391
6.4776540492718455


In [64]:
merged_df = pd.DataFrame({
    'Quito': temp_Quito['AvgTemperature'],
    'Oslo': temp_Oslo['AvgTemperature'],
    'Melbourne': temp_Melbourne['AvgTemperature']
})

print(merged_df)

      Quito  Oslo  Melbourne
0      14.0  -5.0       22.0
1      14.0  -6.0       17.0
2      15.0 -13.0       20.0
3      14.0 -15.0       23.0
4      13.0  -4.0       23.0
...     ...   ...        ...
5839   13.0  -8.0       17.0
5840   14.0 -11.0       19.0
5841   14.0 -18.0       23.0
5842   14.0 -16.0       23.0
5843   14.0 -10.0       24.0

[5844 rows x 3 columns]


In [65]:
def metodoPearson(x, y):
    mean_x = x.sum()/x.count()
    mean_y = y.sum()/y.count()
    
    numerator = np.sum((x - mean_x) * (y - mean_y))
    denominator = np.sqrt(np.sum((x - mean_x)**2) * np.sum((y - mean_y)**2))
    return numerator / denominator

matrizCorr = pd.DataFrame(index=merged_df.columns, columns=merged_df.columns)
for ciudad in merged_df.columns:
    for ciudad2 in merged_df.columns:
        matrizCorr.loc[ciudad, ciudad2] = metodoPearson(merged_df[ciudad], merged_df[ciudad2])

print(matrizCorr)

              Quito      Oslo Melbourne
Quito           1.0  0.066189 -0.053321
Oslo       0.066189       1.0 -0.221326
Melbourne -0.053321 -0.221326       1.0


In [66]:
letters_df = pd.DataFrame(index=merged_df.index, columns=merged_df.columns)

for i in merged_df.columns:
    for j in merged_df.index:
        val = merged_df.loc[j][i]
        if val < 11: letters_df.loc[j, i] = 'F'
        elif val >= 19: letters_df.loc[j, i] = 'C'
        else: letters_df.loc[j, i] = 'T'

print(letters_df)

     Quito Oslo Melbourne
0        T    F         C
1        T    F         T
2        T    F         C
3        T    F         C
4        T    F         C
...    ...  ...       ...
5839     T    F         T
5840     T    F         C
5841     T    F         C
5842     T    F         C
5843     T    F         C

[5844 rows x 3 columns]


In [67]:
ValoresTemp = ['F','T','C']
ValoresCiudad = ['Quito','Oslo','Melbourne']
markovOrdenUno = pd.DataFrame(index=ValoresTemp, columns=ValoresCiudad)

for i in letters_df.columns:
    val_f = 0
    val_t = 0
    val_c = 0

    for j in letters_df.index:
        if letters_df[i][j] == 'F': val_f+=1
        elif letters_df[i][j] == 'T': val_t+=1
        else: val_c+=1

    prob_if = val_f/(val_f+val_t+val_c)
    prob_it = val_t/(val_f+val_t+val_c)
    prob_ic = val_c/(val_f+val_t+val_c)

    markovOrdenUno.loc['F', i] = prob_if
    markovOrdenUno.loc['T', i] = prob_it
    markovOrdenUno.loc['C', i] = prob_ic

print(markovOrdenUno)

      Quito      Oslo Melbourne
F  0.006331  0.685832  0.034223
T  0.992984   0.28063  0.505989
C  0.000684  0.033539  0.459788
